In [ ]:
# Upload the packages
import os
import ee
import geemap

In [ ]:
# Create an ipyleaflet-based interactive map with Geemap
Map = geemap.Map()
Map

In [ ]:
# Draw shapes on the interactive map using the Drawing tools & use it as ROI
# Draw any shapes on the map using the Drawing tools before executing this code block
feature = Map.draw_last_feature
roi = feature.geometry()

In [ ]:
# Upload a collection of images (Sentinel-5p, Tropospheric Formaldehyde) & visualizing it
# Sentinel 5P HCHO column number density (NRT)
collection = (ee.ImageCollection('COPERNICUS/S5P/NRTI/L3_HCHO') \
  .select('tropospheric_HCHO_column_number_density') \
  .filterDate('2020-01-01', '2020-12-31')) 
#.filterBounds(roi))

In [ ]:
# Define the range
val_max = 0.0003
val_min = 0.0
band_viz = {
  'min': val_min,
  'max': val_max,
  'opacity': 1.0,
  'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}

Map.addLayer(collection.median(), band_viz, 'HCHO_2020')

In [ ]:
# Reduce the collection with a median reducer.
collection_median = collection.reduce(ee.Reducer.median())
# Clip the imageCollection 
collection_median_clip = collection_median.clip(roi).unmask()
# Visualize the clipped image
Map.addLayer(collection_median_clip, band_viz, 'HCHO_2020')

In [ ]:
# Select the path & define the filename (change the path with yours)
path = '/Users/...'
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
filename = os.path.join(out_dir, 'S5p.tif')

In [ ]:
# Export the image in a local directory
geemap.ee_export_image(collection_median_clip, 
                       filename=filename, 
                       scale=1000,
                       region=roi, 
                       file_per_band=True)